# Datasets Generations

In this notebook, we will generate the datasets for the experiments. We will generate the following datasets:

- `ds_haar_op` - Dataset of operators uniformly sampled from the Haar measure on the unitary group. With labels indicating if the operator is separable or entangled.
- `ds_haar_obs` - Dataset obtained from the previous one by applying projections on a bell state to each operator.

In [ ]:
# imports
import csv
from qiskit import QuantumCircuit
import numpy as np
from qiskit.quantum_info import random_unitary, Operator, DensityMatrix, schmidt_decomposition

In [ ]:
# generation functions

def gen_2qubit_datapoint():
    """Generate a datapoint for a 2-qubit system.

    To generate the datapoint one of two methods is chosen randomly:
    1. A random 4x4 unitary matrix is generated and applied to a 2-qubit quantum circuit.
    2. Two random 2x2 unitary matrices are generated and their tensor product is applied to a 2-qubit quantum circuit.
    The density matrix of the resulting state is then calculated and the Schmidt rank is used to determine
    if the state is separable or entangled. (If the Schmidt rank is 1, the state is separable.)

    Returns:
        *x, sep (tuple): A tuple containing the coefficients of the unitary matrices and a boolean indicating if the
                         state is separable.
    """
    # randomly choose between two generation methods
    if np.random.rand() >= 0.5:
        # generate a random 4x4 unitary matrix
        unitary_op = random_unitary(4)
    else:
        # generate a separable state by applying two random 2x2 unitary matrices
        Ua = random_unitary(2)
        Ub = random_unitary(2)
        unitary_op = Operator(np.kron(Ua, Ub))
    qc = QuantumCircuit(2)
    qc.unitary(unitary_op, [0, 1])

    # get coefficients of the unitary matrix
    x = unitary_op.data.flatten().tolist()

    # determine entanglement
    rho = DensityMatrix.from_instruction(qc)
    sep = len(schmidt_decomposition(rho, [0])) == 1
    return *x, sep

In [ ]:
# pauli matrices
identity = np.eye(2)
pauliX = np.array([[0, 1], [1, 0]])
pauliY = np.array([[0, -1j], [1j, 0]])
pauliZ = np.array([[1, 0], [0, -1]])


# projectors
projector_1 = 1/4 * (identity + 1/np.sqrt(3) * (pauliX + pauliY + pauliZ))
projector_2 = 1/4 * (identity + 1/np.sqrt(3) * (pauliX - pauliY - pauliZ))
projector_3 = 1/4 * (identity + 1/np.sqrt(3) * (-pauliX + pauliY - pauliZ))
projector_4 = 1/4 * (identity + 1/np.sqrt(3) * (-pauliX - pauliY + pauliZ))


# bell states
def psi_plus():
    """Return the density matrix of the psi plus bell state."""

    ket = (np.kron(np.array([1, 0]), np.array([0, 1])) + np.kron(np.array([0, 1]), np.array([1, 0]))) / np.sqrt(2)
    return np.outer(ket, ket)


def psi_minus():
    """Return the density matrix of the psi minus bell state."""

    ket = (np.kron(np.array([1, 0]), np.array([0, 1])) - np.kron(np.array([0, 1]), np.array([1, 0]))) / np.sqrt(2)
    return np.outer(ket, ket)


# projection
def Pxy(density_matrix, observablex, observabley):
    """The probability of succesfully projecting on a bell state the collective measures a pair of local projections."""

    # Project on the psi minus bell state
    bell_state = psi_minus()
    projection_on_bs = np.kron(np.kron(observablex, bell_state), observabley)

    # Project on the identity to normalize the probability
    projection_on_id = np.kron(np.kron(observablex, np.eye(4)), observabley)

    # Calculate the probability
    rho_T = np.kron(density_matrix, density_matrix)
    up = np.trace(np.dot(rho_T, projection_on_bs))
    down = np.trace(np.dot(rho_T, projection_on_id))

    # the probability is the real part of the trace (complex part should be 0 but numerical errors can occur)
    prob = up / down
    return prob.real

In [ ]:
# transformation functions

def get_density_matrix_from_datapoint(datapoint):
    """Return the density matrix (as array) of a datapoint."""

    operator = get_operator_from_datapoint(datapoint)
    qc = QuantumCircuit(2)
    qc.append(operator, [0, 1])
    return qi.DensityMatrix.from_instruction(qc).data


def get_new_features(datapoint):
    """Obtain a datapoint with the expectation values of the projectors and the label from a (old) datapoint."""
    new_x = []
    density_matrix = get_density_matrix_from_datapoint(datapoint)
    proj_combinations = [(projector_1, projector_1),
                         (projector_2, projector_2),
                         (projector_3, projector_3),
                         (projector_4, projector_4),
                         (projector_1, projector_3),
                         (projector_1, projector_4),
                         (projector_2, projector_4),
                         (projector_1, projector_2),
                         (projector_2, projector_3),
                         (projector_3, projector_4),
                         ]
    # measure the combinations of projectors
    for p1, p2 in proj_combinations:
        new_x.append(Pxy(density_matrix, p1, p2))

    # also return the label
    return *new_x, datapoint[-1]

## Generation

In [ ]:
dataset = [gen_2qubit_datapoint() for _ in range(10_000)]

print(f"Dataset size: {len(dataset)}")
print(f"Entangled states: {sum([1 for datapoint in dataset if datapoint[-1]])}")
print(f"Non-entangled states: {sum([1 for datapoint in dataset if not datapoint[-1]])}")
print(f"len datapoint = {len(dataset[0])}")

In [ ]:
filepath = "dataset.csv"

with open(filepath, "w") as f:
    writer = csv.writer(f)
    writer.writerow(["U11", "U12", "U13", "U14",
                     "U21", "U22", "U23", "U24",
                     "U31", "U32", "U33", "U34",
                     "U41", "U42", "U43", "U44",
                     "sep"])
    writer.writerows(dataset)